# Feliz Mancala
### Caleb Anderson, Aiden Devine

# Libraries, Frameworks, and Update #
We used the aima-python library (https://github.com/aimacode/aima-python.git) for our minimax and alpha-beta pruning algorithm framework. \
For ease of implementation, we copied the functions from the games4e.py file and modified them to fit our Mancala class and it's requirements. 

So far, we have gotten the minmax_decision function working. \
We still need to implement alpha_beta_cutoff_search function which should be straightforward and similar to the min_max function.

## Mancala rules to be followed 
**(there are few modifications from the original game, please read this before writing the code)**

- On every turn, select a pit and distribute its stones in a counter-clockwise direction.
    - If the last stone lands in the player's mancala, in an opponent's pit, or in one of the player's non-empty pits, no further action is taken, and the current player's turn ends.
    - If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side has some stones, collect all those stones, including the one that just landed, and place them into the current player's mancala.

- If either player's pits are entirely empty, the game concludes. The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.

## Small Board (3 Pits of 2 Stones each)

1. **play**: The `play` function allows players to take turns and make moves. The function correctly distributes stones according to the specified game rules.

2. **valid_move**: The `valid_move` function ensures that a player's chosen move is valid.

3. **winning_eval**: The `winning_eval` function determines when the game is over and which player wins. The game ends when any player's pits are all empty. The winner is the player with the most stones in their mancala. If both mancalas have the same number of stones, it's a tie.

The pits are 1-indexed when displaying and picking to make a move.

## Random Player (6 Pits of 4 Stones each)

1. **Random Move Generator**: Define the `random_move_generator` that selects a random pit from the available non-empty pits for the random player. The random player should choose a move based on these criteria. \
Set the 'seed' value to ensure that the generated values remain consistent and reproducible when grading.

You may refer to these links: [How to generate random integers in Python](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/#:~:text=Random%20integer%20values%20can%20be,for%20the%20generated%20integer%20values.), [How to use seed in Python random](https://www.w3schools.com/python/ref_random_seed.asp)


The objective is to play up to **10** moves in total (5 moves by user, 5 moves by random player)

The output submitted should reflect the state of the board and the moves played.

In [239]:
from games4e import minmax_decision, alpha_beta_cutoff_search

In [240]:
import random
import time
from copy import deepcopy
import numpy as np
# random.seed(109)

In [241]:
class Mancala:
    def __init__(self, pits_per_player = 6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
    
    def get_state(self):
        return self.board
    
    def get_actions(self, state):
        """
        Returns a list of all valid moves for a player given a board state
        """
        choices = []
        if self.current_player == 1:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                if state[i] > 0:
                    choices.append(i - self.p1_pits_index[0] + 1)
        else:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                if state[i] > 0:
                    choices.append(i - self.p2_pits_index[0] + 1)

        # print(f"Available Actions for Player {self.current_player}: {choices}")
        return choices
        
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move. Boolean
        """
        if not isinstance(pit, int):
            return False
        elif pit < 1 or pit > self.pits_per_player:
            return False
        
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + (pit - 1)
        else:
            pit_index = self.p2_pits_index[0] + (pit - 1) 
            
        return self.board[pit_index] > 0 
    
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """  
        choices = []
        if self.current_player == 1:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                if self.board[i] > 0:
                    choices.append(i - self.p1_pits_index[0] + 1)
        else:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                if self.board[i] > 0:
                    choices.append(i - self.p2_pits_index[0] + 1)

        return random.choice(choices) if choices else None
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit):
            #print(f"Player {self.current_player} chose pit: {pit}")
            return self.board
        
        if self.winning_eval(self.board) is not None:
            #print("GAME OVER")
            return self.board
        
        self.moves.append((self.current_player, pit))
        #print(f"Player {self.current_player} chose pit: {pit}")
        
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + (pit - 1)
            mancala = self.p1_mancala_index
            opp_mancala = self.p2_mancala_index
            pits = list(range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
            opp_pits = list(range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        else:
            pit_index = self.p2_pits_index[0] + (pit - 1)
            mancala = self.p2_mancala_index
            opp_mancala = self.p1_mancala_index
            pits = list(range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
            opp_pits = list(range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
            
        stones = self.board[pit_index]
        self.board[pit_index] = 0
        curr = pit_index
        
        while stones > 0:
            curr = (curr + 1) % len(self.board)
            
            if curr == opp_mancala:
                continue
            else:
                self.board[curr] += 1
                stones -= 1
            
        if curr == mancala or curr in opp_pits or (curr in pits and self.board[curr] > 1):
            pass
        elif curr in pits and self.board[curr] == 1:
            opp = opp_pits[-(curr - pits[0] + 1)]
            if self.board[opp] > 0:
                self.board[mancala] += self.board[opp] + self.board[curr]
                self.board[opp] = 0
                self.board[curr] = 0
                
        if self.current_player == 1:
            self.current_player = 2
        elif self.current_player == 2:
            self.current_player = 1
        
        self.winning_eval(self.board)
        return self.board
    
    def winning_eval(self, state):
        """
        Function to verify if the game board has reached the winning state.
        If all of either players pits are empty, collect all stones from each players pits and add to their mancala
        Decide the winner based on the mancala total. There are no tie breakers
        """

        p1_empty = all(state[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)) # Check all pits are empty for either player 
        p2_empty = all(state[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        if p1_empty or p2_empty:
            p1_pit_stones = sum(state[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)) # Collect stones in the pits on board
            p2_pit_stones = sum(state[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        
            p1_score = state[self.p1_mancala_index] + p1_pit_stones
            p2_score = state[self.p2_mancala_index] + p2_pit_stones

            if p1_score > p2_score:
                return 1  # Player 1 wins
            elif p2_score > p1_score:
                return 2  # Player 2 wins
            else:
                return 0  # Tie

        return None 


    def utility(self, state, player):
        """
        Utility function. It returns the difference in the mancala pits based on the current player
        """
        p1_pit_stones = sum(state[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)) # Collect stones in the pits on board
        p2_pit_stones = sum(state[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        if player == 1:
            return state[self.p1_mancala_index] + p1_pit_stones - state[self.p2_mancala_index] - p2_pit_stones
        else: 
            return state[self.p2_mancala_index] + p2_pit_stones - state[self.p1_mancala_index] - p1_pit_stones
    


PROJECT PART 3 \
** 6 pits with 4 stones each ** \
On average, the first player wins just over 50% of the time, the second player wins about 43% of the time, and they tie 7% of the time. It takes about 9 moves per game to win.

In [242]:
w = []
mca = []
n = 100

for x in range(n):
    game = Mancala()
    state = game.board
    winner = game.winning_eval(state)
    move_count = 0
    while winner is None:
        move = game.random_move_generator()
        move_count += 1
        if move is not None:
            game.play(move)

        winner = game.winning_eval(state) 
    w.append(winner)
    mca.append(move_count)

p1wp = (w.count(1) / n) * 100
print(f'Player 1 Win Percentage: {p1wp:.2f}%')

p2wp = (w.count(2) / n) * 100
print(f'Player 2 Win Percentage: {p2wp:.2f}%')

p3wp = (w.count(0) / n) * 100
print(f'Tie Percentage:          {p3wp:.2f}%')

print(f'Average moves per game:  {np.mean(mca):.2f}')

Player 1 Win Percentage: 48.00%
Player 2 Win Percentage: 48.00%
Tie Percentage:          4.00%
Average moves per game:  43.84


PROJECT PART 4. \
Minimax AI player that chooses the best move with a variable number of plies and a utility function

In [243]:
def result(game, pit):
    new_game = Mancala()
    new_game.board = game.board.copy()  # Copy the state to avoid reference issues
    new_game.current_player = game.current_player
    new_game.play(pit)
    return new_game  # Return new board state


def MMsearch(game, depth):
    """
    Search game tree to determine best move. Returns (value, move)
    """
    
    inf = np.inf
    player = game.current_player

    def max_value(state, depth):
        if state.winning_eval(state.board) or depth == 0:
            return state.utility(state.board, player), None
        v, move = -inf, None
        for a in state.get_actions(state.board):
            future = result(state, a)
            v2, m = min_value(future, depth - 1)
            if v2 > v or v2 == -inf:  # Modified to return a valid move when "move == None"
                v, move = v2, a
        return v, move

    def min_value(state, depth):
        if state.winning_eval(state.board) or depth == 0:
            return state.utility(state.board, player), None
        v, move = +inf, None
        for a in state.get_actions(state.board):
            future = result(state, a)
            v2, m = max_value(future, depth - 1)
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(game, depth)
    

PROJECT PART 5. RANDOM vs MIN MAX AI \
** 6 pits with 4 stones each ** 

With the random bot versus the MiniMax AI, it wins about 96% of the time with 2% ties when it goes first and 99% when it goes second with 1% ties. When the random goes first, it can win 1% of the time and 2% of the time when it goes second. It takes about 31 moves to finish a game, regardless of who goes first.

The AI is incredibly better than the random player. The most notable reason for this is the decision-making process. The random player chooses a random move from the current board state, and the minimax decides the best move for itself and evaluates the best countermoves. Minimizing the opponents' advantages can dominate a random player that considers no strategy. The minimax player can increase its advantage with more pits and depth.


RAND FIRST: \
Player 1 Win Percentage: 2.00% \
Player 2 Win Percentage: 98.00% \
Tie Percentage:          0.00% \
Average moves per game:  32.95 \
Average time per game:   0.34 seconds

MM FIRST: \
Player 1 Win Percentage: 95.00% \
Player 2 Win Percentage: 4.00% \
Tie Percentage:          1.00% \
Average moves per game:  33.25 \
Average time per game:   0.35 seconds

In [244]:
# PROJECT PART 5. RANDOM vs MIN MAX AI
w = []
mca = []
tavg = []
num_games = 100

for x in range(num_games):
    start = time.time()
    game = Mancala()
    state = game.get_state()
    num_moves = 0
    winner = None

    while winner is None:
        if num_moves % 2 == 1:  # Random player is FIRST if == 0
            rand_move = game.random_move_generator()
            if game.valid_move(rand_move):
                state = game.play(rand_move)
                num_moves += 1
            else:
                print("Invalid move")
                assert False, move
        else:
            _, move = MMsearch(game, 5)
            if move is not None and game.valid_move(move):
                state = game.play(move)
                num_moves += 1
            else:
                print(f'MINBOT CHOSE: {move}  |  {game.board}')
                assert False, move

        winner = game.winning_eval(state) 

    if winner is not None:
        w.append(winner)
        mca.append(num_moves)

        end = time.time()
        tavg.append(end - start)

p1wp = (w.count(1) / num_games) * 100
print(f'Player 1 Win Percentage: {p1wp:.2f}%')

p2wp = (w.count(2) / num_games) * 100
print(f'Player 2 Win Percentage: {p2wp:.2f}%')

p3wp = (w.count(0) / num_games) * 100
print(f'Tie Percentage:          {p3wp:.2f}%')

print(f'Average moves per game:  {np.mean(mca):.2f}')
print(f'Average time per game:   {np.mean(tavg)} seconds')


Player 1 Win Percentage: 97.00%
Player 2 Win Percentage: 3.00%
Tie Percentage:          0.00%
Average moves per game:  33.03
Average time per game:   0.39808518409729005 seconds


PROJECT PART 6. \
Build an AI player that uses Alpha-Beta to choose the best move

In [245]:
def ABsearch(game, depth):
    """Search game tree to determine best move; return (value, move) pair."""
    inf = np.inf
    player = game.current_player

    def max_value(state, alpha, beta, depth):
        if state.winning_eval(state.board) or depth == 0:
            return state.utility(state.board, player), None
        v, move = -inf, None
        for a in state.get_actions(state.board):
            future = result(state, a)
            v2, m = min_value(future, alpha, beta, depth - 1)
            if v2 > v or v2 == -inf:  # had to add the OR check to fix when "move == None"
                v, move = v2, a
            if v >= beta:
                return v, move
            alpha = max(alpha, v)
        return v, move

    def min_value(state, alpha, beta, depth):
        if state.winning_eval(state.board) or depth == 0:
            return state.utility(state.board, player), None
        v, move = +inf, None
        for a in state.get_actions(state.board):
            future = result(state, a)
            v2, m = max_value(future, alpha, beta, depth - 1)
            if v2 < v:
                v, move = v2, a
            if v <= alpha:
                return v, move
            beta = min(beta, v)
        return v, move

    return max_value(game, -inf, inf, depth)
    

PROJECT PART 7. \
** 6 pits with 4 stones each **

The average game takes .036s. The AlphaBeta player wins around 98% of the time.

RANDOM GOES FIRST \
RANDOM Win Percentage:   1.30% \
AB Win Percentage:       98.00% \
Tie Percentage:          0.70% \
Average moves per game:  33.28 \
Average time per game:   0.034 seconds

AB GOES FIRST \
AB Win Percentage:       97.50% \
RANDOM Win Percentage:   1.10% \
Tie Percentage:          1.40% \
Average moves per game:  33.58 \
Average time per game:   0.037 seconds

The AlphaBeta algorithm still destroys the random player but seems slightly more consistent than the Minimax player. The main difference between the two algorithms is the number of nodes searched. The AlphaBeta algorithm searches about half the MiniMax nodes for the same result. With a game this simple and short, the benefits are small, but as the depth increases, they will become significant. Additionally, we see roughly the exact win percentages as before because all that changed was the number of nodes searched. Each game state has an optimal move, and both algorithms can find it, but MiniMax has to search all the nodes to find it where AlphaBeta can prune branches that will not yield worthwhile results.


In [246]:
# PROJECT PART 7.
w = []
mca = []
tavg = []
num_games = 100

for x in range(num_games):
    start = time.time()
    game = Mancala()
    state = game.get_state()  # Initialize the game state
    num_moves = 0
    winner = None

    while winner is None:
        if num_moves % 2 == 0:  # Random player is FIRST if == 0
            rand_move = game.random_move_generator()
            if game.valid_move(rand_move):
                state = game.play(rand_move)
                num_moves += 1
            else:
                print("Invalid move")
                assert False, move
        else:
            _, move = ABsearch(game, 5)
            if move is not None and game.valid_move(move):
                state = game.play(move)
                num_moves += 1
            else:
                print(f'MINBOT CHOSE: {move}  |  {game.board}')
                assert False, move

        winner = game.winning_eval(state) 

    if winner is not None:
        w.append(winner)
        mca.append(num_moves)

        end = time.time()
        tavg.append(end - start)

# Calculations for win percenatages
p1wp = (w.count(1) / num_games) * 100
print(f'Player 1 Win Percentage: {p1wp:.2f}%')

p2wp = (w.count(2) / num_games) * 100
print(f'Player 2 Win Percentage: {p2wp:.2f}%')

p3wp = (w.count(0) / num_games) * 100
print(f'Tie Percentage:          {p3wp:.2f}%')

print(f'Average moves per game:  {np.mean(mca):.2f}')
print(f'Average time per game:   {np.mean(tavg)} seconds')

Player 1 Win Percentage: 2.00%
Player 2 Win Percentage: 97.00%
Tie Percentage:          1.00%
Average moves per game:  34.11
Average time per game:   0.03639524221420288 seconds


PROJECT PART 8. \
RUN AT 10 PLIES

Does increasing the number of plies improve the play for our AI player? Why
or why not?

The first player wins 100% of the time. This is not unexpected as we have implemented depth, the first player to move can find a winning terminal state first. AlphaBeta and Minimax will produce the same quality of answers using the same heuristic, but, AB will just have to search fewer nodes. This means that in everything besides play order, they will produce the same move, but the first player has the advantage of the first move and it is able to win every time.

p == 1 \
MiniMax Win Percentage:   100.00% \
AlphaBeta Win Percentage: 0.00% \
Tie Percentage:           0.00% \
Average moves per game:   54.00 \
Average time per game:    1381.3507509231567 seconds

p == 0 \
AlphaBeta Win Percentage: 100.00% \
MiniMax Win Percentage:   0.00% \
Tie Percentage:           0.00% \
Average moves per game:   54.00 \
Average time per game:    1306.8680639266968 seconds

In [249]:
# PROJECT PART 8.
w = []
mca = []
tavg = []
num_games = 1

for x in range(num_games):
    start = time.time()
    game = Mancala()
    state = game.get_state()
    num_moves = 0
    winner = None
    p = 0 ## 1 or 0

    while winner is None:
        if num_moves % 2 == p:  # AB player is FIRST if == 0
            _, move = ABsearch(game, 10)
            if move is not None and game.valid_move(move):
                state = game.play(move)
                num_moves += 1
            else:
                print(f'AB CHOSE: {move}  |  {game.board}')
                assert False, move
        else:
            _, move = MMsearch(game, 10)
            if move is not None and game.valid_move(move):
                state = game.play(move)
                num_moves += 1
            else:
                print(f'MM CHOSE: {move}  |  {game.board}')
                assert False, move

        winner = game.winning_eval(state) 

    if winner is not None:
        w.append(winner)
        mca.append(num_moves)

        end = time.time()
        tavg.append(end - start)

p1 = 'AlphaBeta' if p == 0 else 'MiniMax'
p2 = 'MiniMax' if p == 0 else 'AlphaBeta'

p1wp = (w.count(1) / num_games) * 100
print(f'{p1} Win Percentage: {p1wp:.2f}%')

p2wp = (w.count(2) / num_games) * 100
print(f'{p2} Win Percentage: {p2wp:.2f}%')

p3wp = (w.count(0) / num_games) * 100
print(f'Tie Percentage:          {p3wp:.2f}%')

print(f'Average moves per game:  {np.mean(mca):.2f}')
print(f'Average time per game:   {np.mean(tavg)} seconds')

AlphaBeta Win Percentage: 100.00%
MiniMax Win Percentage: 0.00%
Tie Percentage:          0.00%
Average moves per game:  54.00
Average time per game:   1306.8680639266968 seconds


In [248]:
# print(f'MAX  |  state: {state}  |   move: {move}  |   a: {a}   |   v: {v}\t|   v2: {v2}   \tactions: {game.get_actions(state)}')